---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [151]:
import pandas as pd
import numpy as np

# train_data = pd.read_csv("train.csv",encoding = 'ISO-8859-1')
# train_data

In [16]:
# train_data.shape

(250306, 34)

In [22]:
# train_data[(train_data['compliance'] == 0) | (train_data['compliance'] == 1)].shape

(159880, 34)

In [15]:
# test_data = pd.read_csv("test.csv", encoding = 'ISO-8859-1')
# test_data

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,violation_description,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status
0,284932,Department of Public Works,"Granberry, Aisha B","FLUELLEN, JOHN A",10041.0,ROSEBERRY,NaN,141,ROSEBERRY,DETROIT,...,Failure to secure City or Private solid waste ...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN
1,285362,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,NaN
2,285361,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Improper placement of Courville container betw...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN
3,285338,Department of Public Works,"Talbert, Reginald","HARABEDIEN, POPKIN",1835.0,CENTRAL,NaN,2246,NELSON,WOODHAVEN,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN
4,285346,Department of Public Works,"Talbert, Reginald","CORBELL, STANLEY",1700.0,CENTRAL,NaN,3435,MUNGER,LIVONIA,...,Violation of time limit for approved container...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN
5,285345,Department of Public Works,"Talbert, Reginald","CORBELL, STANLEY",1700.0,CENTRAL,NaN,3435,MUNGER,LIVONIA,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN
6,285347,Department of Public Works,"Talbert, Reginald","CORBELL, STANLEY",1700.0,CENTRAL,NaN,3435,MUNGER,LIVONIA,...,Inoperable motor vehicle(s) one- or two-family...,Responsible by Default,50.0,20.0,10.0,5.0,0.0,0.0,85.0,NaN
7,285342,Department of Public Works,"Talbert, Reginald","NICKOLA CORPORATION, W & H",1605.0,LIVERNOIS,NaN,1382,WHITEHOUSE CT,ROCHESTER HILLS,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Determination,200.0,20.0,10.0,0.0,0.0,0.0,230.0,NaN
8,285530,Department of Public Works,"Buchanan, Daryl","INTERSTATE INVESTMENT GROUP LL, .",3408.0,BEATRICE,NaN,341,HAMPTON,GILBERT,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,NaN
9,284989,Department of Public Works,"Buchanan, Daryl","YAMAN, BATURAY",8040.0,SARENA,NaN,43494,ELLSWORTH # 20,FREMONT,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,500.0,20.0,10.0,50.0,0.0,0.0,580.0,NaN


In [17]:
# test_data.shape

(61001, 27)

In [27]:
# test_data[test_data['city'] == 'DETROIT'].shape

(26358, 27)

In [32]:
# address_data = pd.read_csv('addresses.csv')
# address_data.head()

,ticket_id,address
0,22056,"2900 tyler, Detroit MI"
1,27586,"4311 central, Detroit MI"
2,22062,"1449 longfellow, Detroit MI"
3,22084,"1441 longfellow, Detroit MI"
4,22093,"2449 churchill, Detroit MI"


In [33]:
# latlon_data = pd.read_csv('latlons.csv')
# latlon_data.head()

,address,lat,lon
0,"4300 rosa parks blvd, Detroit MI 48208",42.346169,-83.079962
1,"14512 sussex, Detroit MI",42.394657,-83.194265
2,"3456 garland, Detroit MI",42.373779,-82.986228
3,"5787 wayburn, Detroit MI",42.403342,-82.957805
4,"5766 haverhill, Detroit MI",42.407255,-82.946295


In [39]:
#join addresses.csv and latlons.csv
# address_data_new = address_data.set_index('address').join(latlon_data.set_index('address'),how = 'left')
# address_data_new.head()

,ticket_id,lat,lon
address,,,
"-11064 gratiot, Detroit MI",328722,42.406935,-82.995599
"-11871 wilfred, Detroit MI",350971,42.411288,-82.993674
"-15126 harper, Detroit MI",344821,42.406402,-82.957525
"0 10th st, Detroit MI",24928,42.325689,-83.064330
"0 10th st, Detroit MI",71887,42.325689,-83.064330


In [40]:
# train_data_new = train_data.set_index('ticket_id').join(address_data_new.set_index('ticket_id'))
# train_data_new.head()

,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,state,...,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance,lat,lon
ticket_id,,,,,,,,,,,,,,,,,,,,,
22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,IL,...,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0,42.390729,-83.124268
27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,MI,...,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0,42.326937,-83.135118
22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,MI,...,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN,42.380516,-83.096069
22084,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","MOROSI, MIKE",1441.0,LONGFELLOW,NaN,5.0,ST. CLAIR,DETROIT,MI,...,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN,42.380570,-83.095919
22093,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","NATHANIEL, NEAL",2449.0,CHURCHILL,NaN,7449.0,CHURCHILL,DETROIT,MI,...,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN,42.145257,-83.208233


In [41]:
# train_data_new.shape

(250306, 35)

In [63]:
# Remove compliance == Nah
# train_data_new = train_data_new[(train_data_new['compliance'] == 0) | (train_data_new['compliance'] == 1)]

In [65]:
# train_data_new.shape

(159880, 35)

In [42]:
# test_data_new = test_data.set_index('ticket_id').join(address_data_new.set_index('ticket_id'))
# test_data_new

,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,state,...,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status,lat,lon
ticket_id,,,,,,,,,,,,,,,,,,,,,
284932,Department of Public Works,"Granberry, Aisha B","FLUELLEN, JOHN A",10041.0,ROSEBERRY,NaN,141,ROSEBERRY,DETROIT,MI,...,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN,42.407581,-82.986642
285362,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,MI,...,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,NaN,42.426239,-83.238259
285361,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,MI,...,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN,42.426239,-83.238259
285338,Department of Public Works,"Talbert, Reginald","HARABEDIEN, POPKIN",1835.0,CENTRAL,NaN,2246,NELSON,WOODHAVEN,MI,...,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN,42.309661,-83.122426
285346,Department of Public Works,"Talbert, Reginald","CORBELL, STANLEY",1700.0,CENTRAL,NaN,3435,MUNGER,LIVONIA,MI,...,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN,42.308830,-83.121116
285345,Department of Public Works,"Talbert, Reginald","CORBELL, STANLEY",1700.0,CENTRAL,NaN,3435,MUNGER,LIVONIA,MI,...,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN,42.308830,-83.121116
285347,Department of Public Works,"Talbert, Reginald","CORBELL, STANLEY",1700.0,CENTRAL,NaN,3435,MUNGER,LIVONIA,MI,...,50.0,20.0,10.0,5.0,0.0,0.0,85.0,NaN,42.308830,-83.121116
285342,Department of Public Works,"Talbert, Reginald","NICKOLA CORPORATION, W & H",1605.0,LIVERNOIS,NaN,1382,WHITEHOUSE CT,ROCHESTER HILLS,MI,...,200.0,20.0,10.0,0.0,0.0,0.0,230.0,NaN,42.313314,-83.108636
285530,Department of Public Works,"Buchanan, Daryl","INTERSTATE INVESTMENT GROUP LL, .",3408.0,BEATRICE,NaN,341,HAMPTON,GILBERT,SC,...,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,NaN,42.261245,-83.160878


In [43]:
# test_data_new.shape

(61001, 28)

In [62]:
# train_data_new['ticket_issued_date'].head()

ticket_id
22056    2004-03-16 11:40:00
27586    2004-04-23 12:30:00
22062    2004-04-26 13:40:00
22084    2004-04-26 13:30:00
22093    2004-04-26 13:00:00
Name: ticket_issued_date, dtype: object

In [61]:
# train_data_new['hearing_date'].head()

ticket_id
22056    2005-03-21 10:30:00
27586    2005-05-06 13:30:00
22062    2005-03-29 10:30:00
22084                    NaN
22093    2005-03-29 10:30:00
Name: hearing_date, dtype: object

In [117]:
# test_data_new['hearing_date'].shape
# test_data_new[~test_data_new['hearing_date'].isnull()].shape

(58804, 28)

In [70]:
# Removal the data of hearing_date is null 
# train_data_new = train_data_new[~train_data_new['hearing_date'].isnull()]

In [67]:
# compute the time
# from datetime import datetime
# def day_gap(hearing_date_str, ticket_issued_date_str):
#     if not hearing_date_str: 
#         return 73
#     hearing_date = datetime.strptime(hearing_date_str, "%Y-%m-%d %H:%M:%S")
#     ticket_issued_date = datetime.strptime(ticket_issued_date_str, "%Y-%m-%d %H:%M:%S")
#     gap = hearing_date - ticket_issued_date
#     return gap.days

In [68]:
# gap = datetime.strptime("2005-05-01 15:00:00", "%Y-%m-%d %H:%M:%S") - datetime.strptime("2004-06-16 12:30:00", "%Y-%m-%d %H:%M:%S")
# gap.days

319

In [73]:
# Add a new columns day_gap
# train_data_new['time_gap'] = train_data_new.apply(lambda row: time_gap(row['hearing_date'], row['ticket_issued_date']), axis=1)

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [75]:
# train_data_new['time_gap'].mean()

72.647410321133961

In [156]:
def blight_model():
    
    from sklearn.ensemble import RandomForestClassifier
#     from datetime import datetime
    from sklearn.metrics import roc_auc_score
#     from sklearn.model_selection import train_test_split
    from sklearn.model_selection import GridSearchCV
    
    # data handling
    train_data = pd.read_csv("train.csv",encoding = 'ISO-8859-1')
    train_data_new = train_data.dropna(how='any',subset=['compliance'])
    test_data = pd.read_csv("test.csv", encoding = 'ISO-8859-1')
    
#     address_data = pd.read_csv('addresses.csv')
#     latlon_data = pd.read_csv('latlons.csv')
    
#     address_data_new = address_data.set_index('address').join(latlon_data.set_index('address'),how = 'left')
#     train_data_new = train_data.set_index('ticket_id').join(address_data_new.set_index('ticket_id'))
#     test_data_new = test_data.set_index('ticket_id').join(address_data_new.set_index('ticket_id'))
    
#     train_data_new = train_data_new[~train_data_new['hearing_date'].isnull()]
#     test_data_new = test_data_new[~test_data_new['hearing_date'].isnull()]
    
    # time method
#     def day_gap(hearing_date_str, ticket_issued_date_str):
#         if not hearing_date_str: 
#             return 73
#         hearing_date = datetime.strptime(hearing_date_str, "%Y-%m-%d %H:%M:%S")
#         ticket_issued_date = datetime.strptime(ticket_issued_date_str, "%Y-%m-%d %H:%M:%S")
#         gap = hearing_date - ticket_issued_date
#         return gap.days

#     # compute day gaps and add a new columns
#     train_data_new['time_gap'] = train_data_new.apply(lambda row: time_gap(row['hearing_date'], row['ticket_issued_date']), axis=1)
#     test_data_new['time_gap'] = test_data_new.apply(lambda row: time_gap(row['hearing_date'], row['ticket_issued_date']), axis=1)
         
    features = ['fine_amount',
               'admin_fee',
               'state_fee',
               'late_fee',
               'discount_amount',
               'judgment_amount']
    
    X_train = train_data_new[features]
    X_test = test_data[features]
    y_train = train_data_new['compliance']
    
    clf = RandomForestClassifier()
    param_values = {'n_estimators':[100],'max_features':[6]}
    grid_clf = GridSearchCV(clf,param_grid = param_values,scoring = 'roc_auc')
    grid_clf.fit(X_train,y_train)
    y_tr_pred = grid_clf.best_estimator_.predict_proba(X_train)
    
    auc_train = roc_auc_score(y_train,y_tr_pred[:,1])
#     print(auc_train)
    
    y_pred = grid_clf.best_estimator_.predict_proba(X_test)
    series = pd.Series(data = y_pred[:,1], index = test_data['ticket_id'], dtype ='float32')
    return series

blight_model()

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


0.758048355901
